In [128]:
import folium
import pandas as pd
import geopandas as gpd
import geoviews as gv
from bokeh.io import output_notebook
from bokeh.plotting import show

In [129]:
GAME_DATA_PATH = "vgsales_clean.csv"
MAP_CENTER = [0, 0]
MAP_ZOOM = 2
EU_LAT = 54.5260
EU_LON = 15.2551
JP_LAT = 36.2048
JP_LON = 138.2529
NA_LAT = 39.30
NA_LON = -94.71
EU_COLOR = 'yellow'
JP_COLOR = 'red'
NA_COLOR = 'blue'
LOC_DATA = {
    'EU_Sales': {'lat': 54.53, 'lon': 15.26, 'color': '#7F96FF',},
    'JP_Sales': {'lat': 36.20, 'lon': 138.25, 'color': '#77FF94',},
    'NA_Sales': {'lat': 39.30, 'lon': -94.71, 'color': '#EF6F6C',},
}

In [130]:
df = pd.read_csv(GAME_DATA_PATH )
print(df.shape)
df.head()

(16291, 11)


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16291 entries, 0 to 16290
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16291 non-null  int64  
 1   Name          16291 non-null  object 
 2   Platform      16291 non-null  object 
 3   Year          16291 non-null  float64
 4   Genre         16291 non-null  object 
 5   Publisher     16291 non-null  object 
 6   NA_Sales      16291 non-null  float64
 7   EU_Sales      16291 non-null  float64
 8   JP_Sales      16291 non-null  float64
 9   Other_Sales   16291 non-null  float64
 10  Global_Sales  16291 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [132]:
df_melted = df.melt(id_vars=['Rank','Genre'], value_vars=['NA_Sales', 'EU_Sales', 'JP_Sales'], var_name='region', value_name='sales')
print(df_melted.shape)
df_melted.head(3)

(48873, 4)


,Rank,Genre,region,sales
0,1,Sports,NA_Sales,41.49
1,2,Platform,NA_Sales,29.08
2,3,Racing,NA_Sales,15.85


In [133]:
total_genre_sales = df_melted.groupby(['Genre','region']).sum().reset_index()
for region, vals in LOC_DATA.items():
    mask_s = total_genre_sales.region == region
    total_genre_sales.loc[mask_s, 'lat'] = vals['lat']
    total_genre_sales.loc[mask_s, 'lon'] = vals['lon']
    total_genre_sales.loc[mask_s, 'color'] = vals['color']
print(total_genre_sales.shape)
total_genre_sales.head()

(36, 7)


,Genre,region,Rank,sales,lat,lon,color
0,Action,EU_Sales,25955792,516.48,54.53,15.26,#7F96FF
1,Action,JP_Sales,25955792,158.65,36.20,138.25,#77FF94
2,Action,NA_Sales,25955792,861.77,39.30,-94.71,#EF6F6C
3,Adventure,EU_Sales,14704318,63.74,54.53,15.26,#7F96FF
4,Adventure,JP_Sales,14704318,51.99,36.20,138.25,#77FF94


In [134]:
# Action data frame

action_df = total_genre_sales.loc[total_genre_sales['Genre']=='Action'].copy()
print(action_df.shape)
action_df.head(3)

(3, 7)


,Genre,region,Rank,sales,lat,lon,color
0,Action,EU_Sales,25955792,516.48,54.53,15.26,#7F96FF
1,Action,JP_Sales,25955792,158.65,36.20,138.25,#77FF94
2,Action,NA_Sales,25955792,861.77,39.30,-94.71,#EF6F6C


In [135]:
# Create a map centered around the average latitude and longitude

my_map=folium.Map(location=MAP_CENTER, zoom_start=MAP_ZOOM)

# Loop through the DataFrame and add CircleMarkers to the map
for idx, row in action_df.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],  # Set the location based on lat and lon
        radius=row['sales'] / 10,  # Size is based on the 'size' column, divided to scale it properly
        color=row['color'],  # Marker color
        fill=True,
        fill_color=row['color'],  # Fill color
        fill_opacity=0.6,
        # popup=f"Size: {row['size']}",  # Add popup with size info
    ).add_to(my_map)

my_map

In [136]:
# Sports data frame

sports_df = total_genre_sales.loc[total_genre_sales['Genre']=='Sports'].copy()
print(sports_df.shape)
sports_df.head(3)

(3, 7)


,Genre,region,Rank,sales,lat,lon,color
30,Sports,EU_Sales,17105195,371.34,54.53,15.26,#7F96FF
31,Sports,JP_Sales,17105195,134.76,36.20,138.25,#77FF94
32,Sports,NA_Sales,17105195,670.09,39.30,-94.71,#EF6F6C


In [137]:
# Create a map centered around the average latitude and longitude

my_map=folium.Map(location=MAP_CENTER, zoom_start=MAP_ZOOM)

# Loop through the DataFrame and add CircleMarkers to the map
for idx, row in sports_df.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],  # Set the location based on lat and lon
        radius=row['sales'] / 10,  # Size is based on the 'size' column, divided to scale it properly
        color=row['color'],  # Marker color
        fill=True,
        fill_color=row['color'],  # Fill color
        fill_opacity=0.6,
        # popup=f"Size: {row['size']}",  # Add popup with size info
    ).add_to(my_map)

my_map

In [138]:
# Shooter sales data frame
shooter_df = total_genre_sales.loc[total_genre_sales['Genre']=='Shooter'].copy()
print(shooter_df.shape)
shooter_df.head(3)

(3, 7)


,Genre,region,Rank,sales,lat,lon,color
24,Shooter,EU_Sales,9399409,310.45,54.53,15.26,#7F96FF
25,Shooter,JP_Sales,9399409,38.18,36.20,138.25,#77FF94
26,Shooter,NA_Sales,9399409,575.16,39.30,-94.71,#EF6F6C


In [139]:
# Create a map centered around the average latitude and longitude

my_map=folium.Map(location=MAP_CENTER, zoom_start=MAP_ZOOM)

# Loop through the DataFrame and add CircleMarkers to the map
for idx, row in shooter_df.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],  # Set the location based on lat and lon
        radius=row['sales'] / 10,  # Size is based on the 'size' column, divided to scale it properly
        color=row['color'],  # Marker color
        fill=True,
        fill_color=row['color'],  # Fill color
        fill_opacity=0.6,
        # popup=f"Size: {row['size']}",  # Add popup with size info
    ).add_to(my_map)
    
my_map